In [1]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import date

In [72]:
team_urls = pd.read_csv('Spiele_Links.csv')
idx = 2
print(team_urls['team'][idx])
url = team_urls['url'][idx]
response = requests.get(url)
df = pd.DataFrame()

B2


In [73]:
#BeautifulSoup HTML-Dokument aus dem Quelltext parsen
soup= BeautifulSoup(response.text, 'html.parser')

In [74]:
#Tabelle aus dem HTML-Dokument extrahieren
table = soup.find('div', {"id":'id-team-matchplan-table'})
#print(table)

In [78]:
team=team_urls['team'][idx]
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)

try:
    for data in table.find_all('tbody'):
        dates = list()
        times = list()
        teams = list()
        rows = data.find_all('tr')
        kw = list()
        for i in range(1,len(rows),3):
            #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
            date = rows[i].find('td', class_='column-date') #rows[i].text
            cleanString = re.sub(r'\ \|',',', date.text)
            cleanString = re.sub(r' ', '', cleanString)
            #removedSpace = re.sub(r'\t', '', removedSpace)
            splitted= cleanString.split(',')
            date_split = splitted[1].split('.')
            datum = datetime.date(int(date_split[2])+2000, int(date_split[1]), int(date_split[0]))
            kw.append(datum.isocalendar()[1])
            dates.append(splitted[1])
            times.append(splitted[2].encode('utf-8').decode('utf-8'))
            teams.append(team)

        df = pd.DataFrame(list(zip(dates,times,teams)),columns=['Datum','Zeit','Team'])
        df.insert(len(df.columns), 'KW', kw)
except:
    print("no data")

no data


In [69]:
if (df.shape[0]>0):
    #Club und Logo auslesen (befindet sich in jeder 3. Zeile ab 3. Zeile)
    for data in table.find_all('tbody'):
        rows = data.find_all('tr')
        club1 = list()
        club2 = list()
        for i in range(2,len(rows),3):
            #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
            #print(rows[i].text)
            removedSpace = re.sub(r'\n', '', rows[i].text)
            removedSpace = re.sub(r'\t', '', removedSpace)
            removedSpace = re.sub(r':', ';', removedSpace)
            splitted = removedSpace.split(';')
            club1.append(splitted[0])
            club2.append(splitted[1][0:-1])


        df.insert(len(df.columns), "Heim", club1)
        df.insert(len(df.columns), "Gast", club2)

In [70]:
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    logos_home = list()
    logos_guest= list()
    for i in range(2,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        logo = rows[i].find_all('div', class_="club-logo") #rows[i].text
        
        logo_link = re.findall(r'//(\S+)\">', str(logo))
        logos_home.append(logo_link[0])
        logos_guest.append(logo_link[1])
        
#found = re.search('AAA(.+?)ZZZ', string).group(1)

    df.insert(len(df.columns), 'Logo Heim', logos_home)
    df.insert(len(df.columns), 'Logo Gast', logos_guest)
    df['Heim'] = df['Heim'].replace(r'SGM.*ABI.*','SGM ABI', regex=True)
    df['Gast'] = df['Gast'].replace(r'SGM.*ABI.*','SGM ABI', regex=True)


In [71]:
df

,Datum,Zeit,Team,KW,Heim,Gast,Logo Heim,Logo Gast
0,06.08.23,10:30,B1,31,SGM ABI,SGM Auenstein-Grossbottwar,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
1,17.09.23,10:30,B1,37,SGM ABI,SGM FV Wüstenrot/​Neuhütten/​Neulautern,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
2,24.09.23,10:30,B1,38,SGM Sülzbach Sulmtal,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
3,01.10.23,10:30,B1,39,SGM ABI,TSG Heilbronn,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
4,08.10.23,10:30,B1,40,TSV Untergruppenbach,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
5,15.10.23,10:30,B1,41,SGM ABI,SGM Auenstein-Grossbottwar,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
6,22.10.23,10:30,B1,42,SGM Lehrensteinsfeld,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
7,29.10.23,10:30,B1,43,SGM ABI,SGM Neckarwestheim /​ Talheim /​ Gemmrigheim I,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
8,19.11.23,10:30,B1,46,TSV Weinsberg,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...


In [51]:
links = table.find_all('a')
print(len(links))
homeLinks = list();
guestLinks = list();
spielLinks = list();
for i in range(2, len(links),6):
    homeLinks.append(links[i].get('href'))
    guestLinks.append(links[i+1].get('href'))
    spielLinks.append(links[i+2].get('href'))
df.insert(len(df.columns), "home_link", homeLinks)
df.insert(len(df.columns), 'guest_link', guestLinks)
df.insert(len(df.columns), 'Spiel', spielLinks)
df

42


,Datum,Zeit,Team,KW,Heim,Gast,Logo Heim,Logo Gast,home_link,guest_link,Spiel
0,16.09.23,13:00,C-J-innen,37,TSV Weinsberg,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/tsv-weinsbe...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/spiel/tsv-weinsberg-sg...
1,23.09.23,14:35,C-J-innen,38,SGM ABI,SV Sülzbach,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/mannschaft/sv-suelzbac...,https://www.fussball.de/spiel/sgm-sc-ilsfeld-a...
2,30.09.23,13:00,C-J-innen,39,SGM ABI,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/spiel/sgm-sc-ilsfeld-a...
3,07.10.23,14:35,C-J-innen,40,Spvgg Oedheim I,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/spvgg-oedhe...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/spiel/spvgg-oedheim-i-...
4,14.10.23,14:35,C-J-innen,41,SGM ABI,TSV Güglingen,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/mannschaft/tsv-gueglin...,https://www.fussball.de/spiel/sgm-sc-ilsfeld-a...
5,21.10.23,13:00,C-J-innen,42,Spvgg Oedheim II,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/spvgg-oedhe...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/spiel/spvgg-oedheim-ii...
6,28.10.23,15:00,C-J-innen,43,SGM ABI,SGM Spfr Widdern,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/mannschaft/sgm-spfr-wi...,https://www.fussball.de/spiel/sgm-sc-ilsfeld-a...


In [52]:
#merge columns to table
if (idx==0):
    df.to_csv('out.csv', index=False, encoding="utf-8")
else:
    df.to_csv('out.csv', mode='a', index=False, header=False, encoding="utf-8")

